In [1]:
import pandas as pd
import numpy as np
import time_series_module as tsm
import time_series_cross_valid as tscv
import time_series_versioning as tsver
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
from importlib import reload

In [2]:
df = pd.read_csv('train.csv')
df['date_time'] = pd.to_datetime(df['date_time'])
date_time_features_names = tsm.get_date_time_features(df, 'date_time', True, True, True, True, True)
date_time_features = df[date_time_features_names].copy()
df.drop(columns = date_time_features_names + ['date_time'], inplace = True)

In [3]:
targets = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']

In [4]:
def get_feature_list(data):
    return [f for f in list(df.columns) if f not in targets and f != 'date_time']

In [5]:
features = get_feature_list(df)

### Ridge regression

In [6]:
def mape(y, y_hat):
    return 100*np.abs(y - y_hat) / y

In [7]:
from sklearn.linear_model import Ridge

In [8]:
ridge_model = Ridge(alpha = .001, normalize = True)
ridge_CrossValid_params = {'train_size' : 2000, 'test_size' : 1, 'min_period' : 0, 'step' : 500}
ridge_ForecastModel_params = {'features' : features, 'date_time' : date_time_features,
                              'prior_lag' : list(range(24)), 'post_lag' : 0, 'new_index' : True}

In [9]:
tscv.run_cv(data = df, targets = targets, horizons = 10, CrossValid_params = ridge_CrossValid_params, 
            ForecastModel_params = ridge_ForecastModel_params, model = ridge_model, metrics = mape)

/home/alexey/notebooks/EPAM/practice_1/time_series_module.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data['lag_' + c + '_' + str(l)] = lagged(data, c, l)


['lag_target_carbon_monoxide_0']


ValueError: cannot reshape array of size 4000 into shape (2000,1)

### Decision trees

### Xgboost